In [4]:
using CLOUD.ConservationLaws
using CLOUD.SpatialDiscretizations
using CLOUD.Solvers
using CLOUD.InitialConditions
using CLOUD.Mesh
using CLOUD.IO
using CLOUD.Analysis
using OrdinaryDiffEq
using TimerOutputs

"Set up paths"
plots_path = "../plots/advection1d_demo/"
results_path = "../results/advection1d_demo/"

"Define the physical problem"
a = 1.0  # advection velocity
A = 1.0  # amplitude
L = 1.0  # domain length
k = 2*π/L  # wave number
T = 1*L/a  # end time

# constant-coefficient linear adection equation with sine wave initial condition
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation(a)
u_exact = initial_condition(initial_data, conservation_law)

"Set discretization parameters"
M = 10  # number of elements
p = 2  # degree of discretization
N_q = 3  # number of quadrature points
reference_approximation=ReferenceApproximation(DGSEM(p), 
    Line(), LGQuadrature())
form = StrongConservationForm()
dt_scale=0.1

"Generate a uniform periodic mesh"
mesh = uniform_periodic_mesh(reference_approximation.reference_element, 
    (0.0,L), M)

"Construct spatial discretization"
spatial_discretization = SpatialDiscretization(mesh, reference_approximation)
    
ode_problem = semidiscretize(conservation_law, 
    spatial_discretization,
    initial_data, form,
    (0.0, T), Eager());

In [5]:
"run the solver"
sol = solve(ode_problem, RK4(), adaptive=false, 
    dt=dt_scale*(L/M)/(a*(2*p+1)), save_everystep=false)
print_timer()
reset_timer!();

 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            32.3s / 0.46%           2.06GiB / 2.96%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 rhs!                   2.00k    148ms   100%  74.0μs   62.5MiB  100%   32.0KiB
   eval residual        20.0k   34.3ms  23.2%  1.72μs   14.0MiB  22.5%     736B
   eval flux diff       20.0k   27.7ms  18.7%  1.38μs   6.72MiB  10.7%     352B
   extrapolate solu...  20.0k   18.7ms  12.6%   934ns   4.89MiB  7.82%     256B
   eval numerical flux  20.0k   14.9ms  10.0%   744ns   18.6MiB  29.8%     976B
   eval flux            20.0k   7.97ms  5.38%   398ns   6.41MiB  10.3%     336B
   gather external ...  20.0k   5.76ms 

In [6]:
"visualize the solution"
plotter = Plotter(spatial_discretization, plots_path)
error_analysis = ErrorAnalysis(spatial_discretization)
visualize(last(sol.u), plotter,
    "final.pdf", exact_solution=u_exact,
     label="U^h(x,t)", label_exact="U(x,t)")
print("L2 error:", calculate_error(error_analysis, last(sol.u), u_exact))

L2 error:0.0006594733046376936